In [12]:
DATA_PATH = '../data/raw/survey_results_public.csv'
EXPORT_PATH = "../data/processed/1_preprocessed_df.pkl"


In [2]:
import numpy as np
import pandas as pd
import re

In [7]:
def split_answers(data_series, delimiter=";"):
    '''
    Split muliple answers in a single string
    to a list of single string each represnting a single answers
    
    Parameters:
    * data series (pd.series): string series with answers
    * delemiter: which string split with defult to (;)
    
    Return: pd.series: if column contains
    '''
    
    def is_splitable(pd_series, delimiter):
        ''' Check if string contain the delimiter and shoud be split - Return boolean'''
        return pd_series.str.contains(delimiter)
    
    def split_answer(pd_series, delimiter):
        ''' function to spilt a single answer'''
        return pd_series.str.split(delimiter)
    #-----------------
    
    # check if multiple answers exist - if none: Return original
    splitable_values = is_splitable(data_series,delimiter)
    if not splitable_values.any():
        return data_series
    
    # Else, split each value to a list
    modified_series = split_answer(data_series,delimiter)
    
    # Replace NAs value with empty list
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])
    
    return modified_series

In [9]:
raw_df = pd.read_csv(DATA_PATH)
df = raw_df.copy()

In [5]:
questionable_columns = ['YearsCode', 'YearsCodePro', 'Age1stCode']


In [10]:
df[questionable_columns[0]] = df[questionable_columns[0]].replace({'Less than 1 year':0, 'More than 50 years':51})
df[questionable_columns[1]] = df[questionable_columns[1]].replace({'Less than 1 year':0, 'More than 50 years':51})
df[questionable_columns[2]] = df[questionable_columns[2]].replace({'Younger than 5 years':4, 'Older than 85':86})

In [11]:
object_cols = df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    df[col] = split_answers(df[col])

In [13]:
df.to_pickle(EXPORT_PATH)
